# Notebook 003 - Cleaning, Transformation and final data loading 

In this Notebook aims to perform the necessary transformations to the data contained in the `candidates_initial_data` for the required visualizations taking into account the analysis performed in the Exploratory Data Analysis Notebook. 

After doing the Exploratory Data Analysis, it was concluded that it is not necessary to delete/impute any records in the dataframe due to outliers, missing or duplicate values. Additionally, the only changes to be made to ensure data compliance were to change the data type of the “application_date” column from object to datetime64 and to standardize the data in the categorical columns to lower case to facilitate data analysis.

Initially, the workshop001 directory is added to the system path so the system can import the scripts package and use the db_connector script to establish the database connection.

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

The necessary libraries for the development of the notebook are imported.

In [2]:
import pandas as pd
import json
from sqlalchemy import create_engine
from scripts.db_connector import get_db_engine

The table is loaded from the database into a dataframe using pandas and a query.

In [3]:
conn = get_db_engine()

In [4]:
query = "SELECT * FROM candidates_initial_data;"
df = pd.read_sql(query, conn)

df

C:\Users\valen\AppData\Local\Temp\ipykernel_1016\2478002064.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


#### Data cleaning

After loading the data we proceed with the defined cleanup:
- Change of the data type of application_date
- Standardization of the columns email, country, seniority, technology

 The data type of “application_date” is changed from object to datetime.

In [5]:
df["application_date"]=df["application_date"].astype("datetime64[ns]")

The values of the columns email, country, seniority, technology are set in lower case.

In [6]:
cols = ["email", "country", "seniority", "technology"]
df[cols] = df[cols].apply(lambda x: x.str.lower())

In [7]:
df.head()

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,norway,2,intern,data engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,panama,10,intern,data engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,belarus,4,mid-level,client success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,eritrea,25,trainee,qa manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,myanmar,13,mid-level,social media community management,9,7


#### Transformations

For the transformations we will take into account the required visualizations of the hired candidates. A candidate is considered hired when his technical interview and code challenge score are equal to or greater than 7. For this a function is written to determine when a candidate is hired or not (1 for hired and 0 for not hired). Then a new column is created and the values calculated by the written function are imputed to it.

In [8]:
def assign_application_status(df):
    if df["code_challenge_score"] >= 7 and df["technical_interview_score"] >= 7:
        return 1
    else:
        return 0


df["application_status"] = df.apply(assign_application_status, axis=1)

df.head()

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,application_status
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,norway,2,intern,data engineer,3,3,0
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,panama,10,intern,data engineer,2,10,0
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,belarus,4,mid-level,client success,10,9,1
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,eritrea,25,trainee,qa manual,7,1,0
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,myanmar,13,mid-level,social media community management,9,7,1


Additionally, a new application_year column is created from the application_date column to facilitate the construction of the hires by year graph required in the dashboard.

In [9]:
df["application_year"] = df["application_date"].dt.year

At last, the columns first_name, last_name, email are eliminated as they are not significant for the dashboard and this reduces the noise in the table that will be used for the visualizations.

In [10]:
df.drop(columns= ["first_name", "last_name", "email"], inplace=True)

Verification of the dataframe after application of the necessary transformations

In [11]:
df.head()

,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,application_status,application_year
0,2021-02-26,norway,2,intern,data engineer,3,3,0,2021
1,2021-09-09,panama,10,intern,data engineer,2,10,0,2021
2,2020-04-14,belarus,4,mid-level,client success,10,9,1,2020
3,2020-10-01,eritrea,25,trainee,qa manual,7,1,0,2020
4,2020-05-20,myanmar,13,mid-level,social media community management,9,7,1,2020


#### Loading of clean data

The last step is to load the clean data using the SQLalchemy library for easy data loading.

In [ ]:
with open ("scripts/credentials.json", "r", encoding="utf-8") as file:
    credentials = json.load(file)

db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]

pg_engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:5432/{db_name}")

df.to_sql('candidates_final_data', pg_engine)

Finally, the data load is verified by running a query on the database.

In [ ]:
query = "SELECT * FROM candidates_final_data limit 10;"
df = pd.read_sql(query, conn)

df

C:\Users\valen\AppData\Local\Temp\ipykernel_13400\2086498342.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,index,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,application_status,application_year
0,0,2021-02-26,norway,2,intern,data engineer,3,3,0,2021
1,1,2021-09-09,panama,10,intern,data engineer,2,10,0,2021
2,2,2020-04-14,belarus,4,mid-level,client success,10,9,1,2020
3,3,2020-10-01,eritrea,25,trainee,qa manual,7,1,0,2020
4,4,2020-05-20,myanmar,13,mid-level,social media community management,9,7,1,2020
5,5,2019-08-17,zimbabwe,8,junior,adobe experience manager,2,9,0,2019
6,6,2018-05-18,wallis and futuna,19,trainee,sales,2,9,0,2018
7,7,2021-12-09,myanmar,1,lead,mulesoft,2,5,0,2021
8,8,2018-03-13,italy,18,lead,social media community management,7,10,1,2018
9,9,2022-04-08,timor-leste,25,lead,devops,2,0,0,2022
